In [8]:
from lxml import etree

input_file = "hangzhou_from_json.net.xml"
output_file = "hangzhou_minimal.net.xml"

TARGET_TLS = "intersection_1_1"

tree = etree.parse(input_file)
root = tree.getroot()

# Tags we care about keeping only for the target intersection
tags_to_filter = ["tlLogic", "junction", "connection", "edge", "lane"]

def is_related(elem):
    id_attr = elem.get("id")
    from_attr = elem.get("from")
    to_attr = elem.get("to")
    tl_attr = elem.get("tl")
    
    return (
        (id_attr and TARGET_TLS in id_attr) or
        (from_attr and TARGET_TLS in from_attr) or
        (to_attr and TARGET_TLS in to_attr) or
        (tl_attr and tl_attr == TARGET_TLS)
    )

# Step 1: Filter everything unrelated
for elem in list(root):
    if elem.tag in tags_to_filter and not is_related(elem):
        root.remove(elem)

# Step 2: Save the new file
tree.write(output_file, pretty_print=True, xml_declaration=True, encoding="UTF-8")
print(f"✅ Cleaned XML written to: {output_file}")


✅ Cleaned XML written to: hangzhou_minimal.net.xml


In [2]:
pip install lxml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 15.5 MB/s eta 0:00:0031m12.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
from lxml import etree

# Load the original route file
input_file = "hangzhou_from_json.rou.xml"
output_file = "hangzhou_minimal.rou.xml"

# Define valid edge IDs related to intersection_1_1
valid_edges = {
    "road_1_0_1", "road_1_1_0", "road_1_1_1", "road_1_1_2", "road_1_1_3",
    "road_1_2_3", "road_0_1_0", "road_2_1_2"
}

# Parse the XML
tree = etree.parse(input_file)
root = tree.getroot()

# Loop through vehicle elements and filter
for vehicle in root.findall("vehicle"):
    route = vehicle.find("route")
    if route is not None:
        edge_list = route.get("edges").split()
        if any(edge.split("_")[0:4] and "_".join(edge.split("_")[0:4]) not in valid_edges for edge in edge_list):
            root.remove(vehicle)

# Save the cleaned route file
tree.write(output_file, pretty_print=True, xml_declaration=True, encoding="UTF-8")
print("✅ Cleaned route file saved:", output_file)


✅ Cleaned route file saved: hangzhou_minimal.rou.xml


In [5]:
from lxml import etree

# Paths to your files
net_path = "hangzhou_minimal.net.xml"
rou_path = "hangzhou_minimal.rou.xml"

# 🧠 Load and parse both XML files
net_tree = etree.parse(net_path)
rou_tree = etree.parse(rou_path)

# ✅ Get all valid edge IDs from the network
valid_edges = set()
for edge in net_tree.xpath("//edge"):
    edge_id = edge.get("id")
    if edge_id and not edge_id.startswith(":"):  # Skip internal edges
        valid_edges.add(edge_id)

print(f"✅ Found {len(valid_edges)} valid edges in the network.")

# 🚨 Check each route in the .rou.xml
invalid_routes = []
for route in rou_tree.xpath("//route"):
    vehicle_id = route.getparent().get("id", "unknown")  # try to grab the vehicle/container ID
    route_edges = route.get("edges", "").split()
    for edge in route_edges:
        if edge not in valid_edges:
            print(f"❌ Vehicle '{vehicle_id}' uses unknown edge: '{edge}'")
            invalid_routes.append((vehicle_id, edge))

if not invalid_routes:
    print("🎉 All route edges are valid!")
else:
    print(f"\n🚨 Found {len(invalid_routes)} invalid edge references in the route file.")


✅ Found 8 valid edges in the network.
🎉 All route edges are valid!


In [12]:
from lxml import etree

net_path = "hangzhou_minimal.net.xml"
output_path = "hangzhou_minimal_surgical.net.xml"
target_tls = "intersection_1_1"

tree = etree.parse(net_path)
root = tree.getroot()

# 🕵️ Step 1: Find all edges connected to intersection_1_1
valid_edges = set()
for edge in root.findall("edge"):
    if edge.get("function") == "internal":
        continue  # Skip internal edges
    if edge.get("from") == target_tls or edge.get("to") == target_tls:
        valid_edges.add(edge.get("id"))

print(f"✅ Found {len(valid_edges)} edges connected to {target_tls}: {list(valid_edges)}")

# 🧼 Step 2: Remove unrelated edges
for edge in root.findall("edge"):
    if edge.get("function") != "internal" and edge.get("id") not in valid_edges:
        root.remove(edge)

# 🎯 Step 3: Remove unrelated junctions
for junction in root.findall("junction"):
    if junction.get("id") != target_tls:
        root.remove(junction)

# ✂️ Step 4: Remove connections not tied to valid edges
for conn in root.findall("connection"):
    from_edge = conn.get("from")
    to_edge = conn.get("to")
    if from_edge not in valid_edges and to_edge not in valid_edges:
        root.remove(conn)

print("✅ Stripped network to only include elements tied to:", target_tls)

# 💾 Save the new cleaned file
tree.write(output_path, pretty_print=True, encoding="UTF-8", xml_declaration=True)
print("💾 Saved cleaned net to", output_path)


✅ Found 8 edges connected to intersection_1_1: ['road_1_0_1', 'road_1_1_0', 'road_1_1_2', 'road_1_1_3', 'road_1_1_1', 'road_2_1_2', 'road_1_2_3', 'road_0_1_0']
✅ Stripped network to only include elements tied to: intersection_1_1
💾 Saved cleaned net to hangzhou_minimal_surgical.net.xml


In [7]:
from lxml import etree

# Load net file
tree = etree.parse("hangzhou_minimal_cleaned.net.xml")
root = tree.getroot()

# Gather all existing edge/lane IDs
all_edges = set(e.get("id") for e in root.findall("edge") if not e.get("id").startswith(":"))
all_lanes = set()
for edge in root.findall("edge"):
    for lane in edge.findall("lane"):
        all_lanes.add(lane.get("id"))

print(f"✅ Existing edges: {len(all_edges)}, lanes: {len(all_lanes)}")

# Remove invalid <connection> elements
connection_count = 0
for conn in root.findall("connection"):
    from_edge = conn.get("from")
    to_edge = conn.get("to")
    from_lane = conn.get("fromLane")
    to_lane = conn.get("toLane")

    if from_edge not in all_edges or to_edge not in all_edges:
        root.remove(conn)
        connection_count += 1
        continue

    # Optional: check lane IDs too
    from_lane_id = f"{from_edge}_{from_lane}" if from_lane else None
    to_lane_id = f"{to_edge}_{to_lane}" if to_lane else None
    if from_lane_id and from_lane_id not in all_lanes:
        root.remove(conn)
        connection_count += 1
    elif to_lane_id and to_lane_id not in all_lanes:
        root.remove(conn)
        connection_count += 1

print(f"🧹 Removed {connection_count} invalid <connection> elements.")

# Save it back
tree.write("hangzhou_minimal_cleaned_fixed.net.xml", pretty_print=True, xml_declaration=True, encoding="UTF-8")


✅ Existing edges: 0, lanes: 28
🧹 Removed 48 invalid <connection> elements.


In [13]:
from lxml import etree

net_edges = {'road_1_0_1', 'road_1_1_0', 'road_1_1_2', 'road_1_1_3', 'road_1_1_1', 'road_2_1_2', 'road_1_2_3', 'road_0_1_0'}
route_tree = etree.parse("hangzhou_minimal.rou.xml")
routes = route_tree.findall("route")

invalid = []
for route in routes:
    edges = route.get("edges", "").split()
    for edge in edges:
        if edge not in net_edges:
            invalid.append(edge)

if invalid:
    print("🚨 Invalid edges found in route file:", invalid)
else:
    print("✅ All route edges are valid!")


✅ All route edges are valid!


In [14]:
from lxml import etree

full_net_path = "hangzhou_from_json.net.xml"
surgical_net_path = "hangzhou_minimal_surgical.net.xml"
output_path = "hangzhou_minimal_surgical_fixed.net.xml"

# Load full and surgical network XMLs
full_tree = etree.parse(full_net_path)
full_root = full_tree.getroot()

surgical_tree = etree.parse(surgical_net_path)
surgical_root = surgical_tree.getroot()

# Collect all node IDs referenced in <edge> elements of the surgical net
used_nodes = set()
for edge in surgical_root.findall("edge"):
    from_node = edge.get("from")
    to_node = edge.get("to")
    if from_node:
        used_nodes.add(from_node)
    if to_node:
        used_nodes.add(to_node)

# Add back the corresponding <junction> tags from the full net
full_junctions = {j.get("id"): j for j in full_root.findall("junction")}
for node_id in used_nodes:
    if node_id in full_junctions:
        surgical_root.append(full_junctions[node_id])

# Save the fixed network
surgical_tree.write(output_path, pretty_print=True, xml_declaration=True, encoding="UTF-8")
print("✅ Wrote:", output_path)


✅ Wrote: hangzhou_minimal_surgical_fixed.net.xml


In [15]:
from lxml import etree

net_path = "hangzhou_minimal_surgical.net.xml"
out_path = "hangzhou_minimal_surgical_fixed.net.xml"

tree = etree.parse(net_path)
root = tree.getroot()

# Only care about this one lonely intersection
valid_tls = {"intersection_1_1"}
valid_nodes = set(valid_tls)

# Step 1: Keep only relevant edges
edges_to_keep = set()
for edge in root.findall("edge"):
    from_node = edge.get("from")
    to_node = edge.get("to")
    if from_node in valid_nodes or to_node in valid_nodes:
        edges_to_keep.add(edge.get("id"))

# Step 2: Clean <edge> tags
for edge in root.findall("edge"):
    if edge.get("id") not in edges_to_keep:
        root.remove(edge)

# Step 3: Clean <connection> tags
for conn in root.findall("connection"):
    if conn.get("from") not in edges_to_keep and conn.get("to") not in edges_to_keep:
        root.remove(conn)

# Step 4: Clean <junction> tags
for junction in root.findall("junction"):
    if junction.get("id") not in valid_nodes:
        root.remove(junction)

# Step 5: Clean <tlLogic> (keep only one!)
seen_tls = False
for logic in root.findall("tlLogic"):
    if logic.get("id") == "intersection_1_1":
        if not seen_tls:
            seen_tls = True
        else:
            root.remove(logic)
    else:
        root.remove(logic)

# Step 6: Clean <connection> with broken junction references
for conn in root.findall("connection"):
    if conn.get("from") not in edges_to_keep or conn.get("to") not in edges_to_keep:
        root.remove(conn)

tree.write(out_path, pretty_print=True, xml_declaration=True, encoding="UTF-8")
print(f"✅ Cleaned and saved to: {out_path}")


✅ Cleaned and saved to: hangzhou_minimal_surgical_fixed.net.xml


In [17]:
from lxml import etree

input_file = "edges.xml"
output_file = "edges.xml"

# Parse the messy file
parser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(input_file, parser)
root = tree.getroot()

# Pretty print and write to a new file
tree.write(output_file, pretty_print=True, xml_declaration=True, encoding="UTF-8")

print(f"✅ Cleaned and saved to {output_file}")


✅ Cleaned and saved to edges.xml


In [18]:
from lxml import etree

input_file = "edges.xml"
output_file = "edges.xml"

# Load XML
parser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(input_file, parser)
root = tree.getroot()

for edge in root.findall("edge"):
    lanes = edge.findall("lane")
    if lanes:
        edge.set("numLanes", str(len(lanes)))
        edge.set("speed", "13.9")  # You can customize this value

# Save the cleaned-up version
tree.write(output_file, pretty_print=True, xml_declaration=True, encoding="UTF-8")

print(f"✅ Updated edge definitions and saved to: {output_file}")


✅ Updated edge definitions and saved to: edges.xml


In [19]:
import xml.etree.ElementTree as ET

# FILE PATHS — change as needed
net_file = "hangzhou.net.xml"
rou_file = "hangzhou_from_json.rou.xml"

# Step 1: Extract all valid edge IDs from the .net.xml
net_tree = ET.parse(net_file)
net_root = net_tree.getroot()
valid_edges = {edge.get("id") for edge in net_root.findall("edge")}

print(f"✅ Found {len(valid_edges)} edges in network.")

# Step 2: Check if each route only uses valid edges
rou_tree = ET.parse(rou_file)
rou_root = rou_tree.getroot()

errors_found = 0
for vehicle in rou_root.findall("vehicle"):
    route = vehicle.find("route")
    if route is not None:
        edge_ids = route.get("edges", "").split()
        for edge_id in edge_ids:
            if edge_id not in valid_edges:
                print(f"❌ Invalid edge in route: '{edge_id}' for vehicle '{vehicle.get('id')}'")
                errors_found += 1

if errors_found == 0:
    print("🎉 All route edges are valid!")
else:
    print(f"⚠️ Found {errors_found} invalid route edge references.")


✅ Found 480 edges in network.
🎉 All route edges are valid!


In [22]:
import xml.etree.ElementTree as ET

def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        for child in elem:
            indent(child, level+1)
        if not child.tail or not child.tail.strip():
            child.tail = i
    if level and (not elem.tail or not elem.tail.strip()):
        elem.tail = i
    return elem

# Input/output paths
input_file = "hangzhou.rou.xml"
output_file = "hangzhou.rou.xml"


tree = ET.parse(input_file)
root = tree.getroot()

# Pretty format the XML
indent(root)

tree.write(output_file, encoding="utf-8", xml_declaration=True)
print(f"✅ Cleaned and saved: {output_file}")


✅ Cleaned and saved: hangzhou.rou.xml


In [23]:
import xml.etree.ElementTree as ET

rou_file = "hangzhou.rou.xml"
tree = ET.parse(rou_file)
root = tree.getroot()

# Separate header tags from vehicles
header = []
vehicles = []

for elem in root:
    if elem.tag == "vehicle":
        vehicles.append(elem)
    else:
        header.append(elem)

# Sort vehicles by depart time
vehicles.sort(key=lambda v: float(v.attrib.get("depart", 0)))

# Rebuild the XML
root.clear()
for elem in header + vehicles:
    root.append(elem)

# Write back to file
tree.write("hangzhou.rou.sorted.xml", encoding="utf-8", xml_declaration=True)
print("✅ Vehicles sorted by depart time → saved to 'hangzhou.rou.sorted.xml'")


✅ Vehicles sorted by depart time → saved to 'hangzhou.rou.sorted.xml'
